#1. 필수 라이브러리

In [ ]:
!pip install cohere
!pip install tiktoken
!pip install openai
!pip install langchain
!pip install langchain_experimental
!pip install google-search-results

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent, Tool
from langchain import LLMMathChain
from langchain_experimental.tools.python.tool import PythonAstREPLTool

import pandas as pd
import os

#2.OPENAI/SERPAPI KEY Setting

In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.environ["SERPAPI_API_KEY"] = "SERPAPI_API_KEY"

#3.dashboard_defect_rate data load(Pandas Dataframe)

In [ ]:
df = pd.read_csv("CSV_FILE_PATH/CSV_FILE.csv")

python = PythonAstREPLTool(locals={"df": df}) # set access of python_repl tool to the dataframe
df_columns = df.columns.to_list() # print column names of df

#4.Tool Setting

In [ ]:
tools = [
    Tool(
        name = "dashboard_defect_rate_analysis",
        func=python.run,
        description = f"""
        Useful for when you need to answer questions about defect rate data stored in pandas dataframe 'df_columns'.
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: {df_columns}

        <user>: 질문 프롬프트 예시
        <assistant>: df.groupby('컬럼명')['깂'].sum()
        <assistant>: 답변은 이렇습니다  (ㅇ 예시)
                      BU_CD SUM(BU_CD)
                      CR    26.15
                      LP     7.21
                      QT    12.50
        """
    )
]

#5.Memory Setting/Agent arguments Setting

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(memory_key="memory", return_messages=True)
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    'prefix': f'You are an expert in checking the defect rate.. You have access to the following tools:'
}



#7.LLM Agent Setting

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# initialize the LLM agent
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         agent_kwargs=agent_kwargs,
                         memory=memory,
                         max_execution_time=1,
                         early_stopping_method="generate",
                         )

#8. Query

In [ ]:
agent.run("사업부별 월별 불량률은 얼마인가?")